### Decision tree Estimator on Analyzing a all_vars_for_zeroinf_analysis data 

- This Python notebook demonstrates creating an ML Pipeline to preprocess a dataset, train a Machine Learning model, and make predictions.

- Data: The dataset contains all_vars_for_zeroinf_analysis  

- Goal: We want to learn to predict confirmed deaths  from information such as 

- Approach: We will use Spark ML Pipelines, which help users piece together parts of a workflow such as feature processing and model training. We will also demonstrate model selection (a.k.a. hyperparameter tuning) using Cross Validation in order to fine-tune and improve our ML model.

In [2]:
%sh 
pip install mleap

Requirement already satisfied: mleap in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (0.8.1)
Requirement already satisfied: argparse>=1.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.4.0)
Requirement already satisfied: nose-exclude>=0.5.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (0.5.0)
Requirement already satisfied: scipy>=0.13.0b1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.2.1)
Requirement already satisfied: numpy>=1.8.2 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.18.5)
Requirement already satisfied: six>=1.10.0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.12.0)
Requirement already satisfied: pandas>=0.18.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (1.0.4)
Requirement already satisfied: scikit-learn>=0.18.dev0 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from mleap) (0.20.3)
Requirement already satisfied: nose in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from nose-exclude>=0.5.0->mleap) (1.3.7)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from pandas>=0.18.1->mleap) (2.8.0)
Requirement already satisfied: pytz>=2017.2 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from pandas>=0.18.1->mleap) (2018.9)

In [3]:
from pyspark.sql.functions import log

In [4]:
%sh ls /dbfs/FileStore/tables/

all_vars_for_analysis.csv
all_vars_for_zeroinf_analysis.csv
Feature_Importances_predicting_deaths-d1408.csv
Final_Features_for_analysis-2c971.csv
Final_Features_for_analysis.csv
health.csv
health_factors_behab.csv
health_factors.csv
Health_Professional_Shortage_Areas__Primary_Care__by_County-c0119.csv
housing.csv
Lockdown_Join_with_normaliztion_factor-ed399.csv
merged.csv
merged_dataset.xlsx
merged_emp.csv
merged_source.csv
States_Lock_down_Lengh-8d3af.csv
States_Lockdown_Lengh.csv
testing_by_state-971a2.csv
Train_X.csv
us_counties.csv
us_counties-e349f.csv
X_test.csv
X_train.csv
y_test.csv
y_train.csv

In [5]:
# File location and type
file_location = "/FileStore/tables/all_vars_for_zeroinf_analysis.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

## import all_vars_for_zeroinf_analysis  file 
# The applied options are for CSV files. For other file types, these will be ignored.
zeroinf_analysis = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [6]:
#display(zeroinf_analysis)

In [7]:
zeroinf_analysis.count()

Out[31]: 2890

In [8]:
zeroinf_analysis.dtypes

Out[32]: [('county', 'string'),
 ('confirmed_cases', 'int'),
 ('confirmed_deaths', 'int'),
 ('state', 'string'),
 ('length_of_lockdown', 'int'),
 ('cases', 'double'),
 ('deaths', 'double'),
 ('POP_ESTIMATE_2018', 'int'),
 ('total_state_pop', 'int'),
 ('Active.Physicians.per.100.000.Population..2018..AAMC.', 'double'),
 ('Total.Active.Patient.Care.Physicians.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Active.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Active.Patient.Care.Primary.Care.Physicians.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Active.General.Surgeons.per.100.000.Population..2018..AAMC.', 'double'),
 ('Active.Patient.Care.General.Surgeons.per.100.000.Population..2018..AAMC.',
 'double'),
 ('Percentage.of.Active.Physicians.Who.Are.Female..2018..AAMC.', 'double'),
 ('Percentage.of.Active.Physicians.Who.Are.International.Medical.Graduates..IMGs...2018..AAMC.',
 'double'),
 ('Percentage.of.Active.Physicians.Who.Are.Age.60.or.Older..2018..AAMC.',
 'double'),
 ('MD.and.DO.Student.Enrollment.per.100.000.Population..AY.2018.2019..AAMC.',
 'double'),
 ('Student.Enrollment.at.Public.MD.and.DO.Schools.per.100.000.Population..AY.2018.2019..AAMC.',
 'double'),
 ('Percentage.Change.in.Student.Enrollment.at.MD.and.DO.Schools..2008.2018..AAMC.',
 'double'),
 ('Percentage.of.MD.Students.Matriculating.In.State..AY.2018.2019..AAMC.',
 'double'),
 ('Total.Residents.Fellows.in.ACGME.Programs.per.100.000.Population.as.of.December.31..2018..AAMC.',
 'double'),
 ('Total.Residents.Fellows.in.Primary.Care.ACGME.Programs.per.100.000.Population.as.of.Dec..31..2018..AAMC.',
 'double'),
 ('Percentage.of.Residents.in.ACGME.Programs.Who.Are.IMGs.as.of.December.31..2018..AAMC.',
 'double'),
 ('Ratio.of.Residents.and.Fellows..GME..to.Medical.Students..UME...AY.2017.2018..AAMC.',
 'double'),
 ('Percent.Change.in.Residents.and.Fellows.in.ACGME.Accredited.Programs..2008.2018..AAMC.',
 'double'),
 ('Percentage.of.Physicians.Retained.in.State.from.Undergraduate.Medical.Education..UME...2018..AAMC.',
 'double'),
 ('All.Specialties..AAMC.', 'double'),
 ('State.Local.Government.hospital.beds.per.1000.people..2019.', 'double'),
 ('Non.profit.hospital.beds.per.1000.people..2019.', 'double'),
 ('For.profit.hospital.beds.per.1000.people..2019.', 'double'),
 ('Total.hospital.beds.per.1000.people..2019.', 'double'),
 ('Total.nurse.practitioners..2019.', 'double'),
 ('Total.physician.assistants..2019.', 'double'),
 ('Total.Hospitals..2019.', 'double'),
 ('Total.Primary.Care.Physicians..2019.', 'double'),
 ('Surgery.specialists..2019.', 'double'),
 ('Emergency.Medicine.specialists..2019.', 'double'),
 ('Total.Specialist.Physicians..2019.', 'double'),
 ('ICU.Beds', 'double'),
 ('pop_fraction', 'double'),
 ('Length.of.Life.rank', 'double'),
 ('Quality.of.Life.rank', 'double'),
 ('Health.Behaviors.rank', 'double'),
 ('Clinical.Care.rank', 'double'),
 ('Social...Economic.Factors.rank', 'double'),
 ('Physical.Environment.rank', 'double'),
 ('Adult.smoking.percentage', 'int'),
 ('Adult.obesity.percentage', 'int'),
 ('Excessive.drinking.percentage', 'int'),
 ('Population.per.sq.mile', 'double'),
 ('House.per.sq.mile', 'double'),
 ('Share.of.Tests.with.Positive.COVID.19.Results', 'double'),
 ('Number.of.Tests.with.Results.per.1.000.Population', 'double')]

In [9]:
new_column_name_list= list(map(lambda x: x.replace(".", "_"), zeroinf_analysis.columns))

zeroinf_analysis = zeroinf_analysis.toDF(*new_column_name_list)

In [10]:
zeroinf_analysis.where( zeroinf_analysis['confirmed_deaths'].isNull() ).count()

Out[34]: 0

In [11]:
zeroinf_analysis.where( zeroinf_analysis['pop_fraction'].isNull() ).count()

Out[35]: 0

In [12]:
def count_nulls(df):
    null_counts = []          #make an empty list to hold our results
    for col in df.dtypes:     #iterate through the column data types we saw above, e.g. ('C0', 'bigint')
        cname = col[0]        #splits out the column name, e.g. 'C0'    
        ctype = col[1]        #splits out the column type, e.g. 'bigint'
        if ctype != 'string': #skip processing string columns for efficiency (can't have nulls)
            nulls = df.where( df[cname].isNull() ).count()
            result = tuple([cname, nulls])  #new tuple, (column name, null count)
            null_counts.append(result)      #put the new tuple in our result list
    return null_counts

null_counts = count_nulls(zeroinf_analysis)

In [13]:
null_counts

Out[37]: [('confirmed_cases', 0),
 ('confirmed_deaths', 0),
 ('length_of_lockdown', 0),
 ('cases', 0),
 ('deaths', 0),
 ('POP_ESTIMATE_2018', 0),
 ('total_state_pop', 0),
 ('Active_Physicians_per_100_000_Population__2018__AAMC_', 0),
 ('Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_',
 0),
 ('Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_', 0),
 ('Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_',
 0),
 ('Active_General_Surgeons_per_100_000_Population__2018__AAMC_', 0),
 ('Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_',
 0),
 ('Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_', 0),
 ('Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_',
 0),
 ('Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_', 0),
 ('MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_',
 0),
 ('Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_',
 0),
 ('Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_',
 0),
 ('Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_', 0),
 ('Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_',
 0),
 ('Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_',
 0),
 ('Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_',
 0),
 ('Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_',
 0),
 ('Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_',
 0),
 ('Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_',
 0),
 ('All_Specialties__AAMC_', 0),
 ('State_Local_Government_hospital_beds_per_1000_people__2019_', 0),
 ('Non_profit_hospital_beds_per_1000_people__2019_', 0),
 ('For_profit_hospital_beds_per_1000_people__2019_', 0),
 ('Total_hospital_beds_per_1000_people__2019_', 0),
 ('Total_nurse_practitioners__2019_', 0),
 ('Total_physician_assistants__2019_', 0),
 ('Total_Hospitals__2019_', 0),
 ('Total_Primary_Care_Physicians__2019_', 0),
 ('Surgery_specialists__2019_', 0),
 ('Emergency_Medicine_specialists__2019_', 0),
 ('Total_Specialist_Physicians__2019_', 0),
 ('ICU_Beds', 0),
 ('pop_fraction', 0),
 ('Length_of_Life_rank', 0),
 ('Quality_of_Life_rank', 0),
 ('Health_Behaviors_rank', 0),
 ('Clinical_Care_rank', 0),
 ('Social___Economic_Factors_rank', 0),
 ('Physical_Environment_rank', 0),
 ('Adult_smoking_percentage', 0),
 ('Adult_obesity_percentage', 0),
 ('Excessive_drinking_percentage', 0),
 ('Population_per_sq_mile', 0),
 ('House_per_sq_mile', 0),
 ('Share_of_Tests_with_Positive_COVID_19_Results', 0),
 ('Number_of_Tests_with_Results_per_1_000_Population', 0)]

In [14]:
zeroinf_analysis_described = zeroinf_analysis.describe()
zeroinf_analysis_described.show()

+-------+---------+------------------+-----------------+-------+------------------+-----------------+-------------------+------------------+--------------------+-----------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------+-------------------------------------------------------------------------------+-----------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+----------------------+-----------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------------+--------------------------------+---------------------------------+----------------------+------------------------------------+--------------------------+-------------------------------------+----------------------------------+-------------------+--------------------+-------------------+--------------------+---------------------+------------------+------------------------------+-------------------------+------------------------+------------------------+-----------------------------+----------------------+-----------------+---------------------------------------------+-------------------------------------------------+
summary| county| confirmed_cases| confirmed_deaths| state|length_of_lockdown| cases| deaths| POP_ESTIMATE_2018| total_state_pop|Active_Physicians_per_100_000_Population__2018__AAMC_|Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_General_Surgeons_per_100_000_Population__2018__AAMC_|Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_|MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_|Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_|Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_|Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_|Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_|Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_|Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_|Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_|Percent_Change_in_Residents_and_Fellows_in_ACGME_Accre

In [15]:
from pyspark.sql.functions import skewness, kurtosis
from pyspark.sql.functions import var_pop, var_samp, stddev, stddev_pop, sumDistinct, ntile
zeroinf_analysis.select(skewness('confirmed_deaths')).show()


+--------------------------+
skewness(confirmed_deaths)|
+--------------------------+
 9.791804720559224|
+--------------------------+

In [16]:
from pyspark.sql import Row

columns = zeroinf_analysis_described.columns  #list of column names
funcs   = [skewness, kurtosis]  #list of functions we want to include (imported earlier)
fnames  = ['skew', 'kurtosis']  #a list of strings describing the functions in the same order

def new_item(func, column):
    """
    This function takes in an aggregation function and a column name, then applies the aggregation to the
    column, collects it and returns a value.  The value is in string format despite being a number, 
    because that matches the output of describe.
    """
    return str(zeroinf_analysis.select(func(column)).collect()[0][0])

new_data = []
for func, fname in zip(funcs, fnames):
    row_dict = {'summary':fname}  #each row object begins with an entry for "summary"
    for column in columns[1:]:
        row_dict[column] = new_item(func, column)
    new_data.append(Row(**row_dict))  #using ** tells Python to unpack the entries of the dictionary
    
print(new_data)

[Row(Active_General_Surgeons_per_100_000_Population__2018__AAMC_='2.006230069948525', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='1.65641771563768', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='1.096630292190715', Active_Physicians_per_100_000_Population__2018__AAMC_='2.0468748853313765', Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='1.306654476141262', Adult_obesity_percentage='-0.1002967550395719', Adult_smoking_percentage='0.7847910686933075', All_Specialties__AAMC_='2.0470977928344607', Clinical_Care_rank='1.6971082547615677', Emergency_Medicine_specialists__2019_='1.536501239473279', Excessive_drinking_percentage='0.16177195733968358', For_profit_hospital_beds_per_1000_people__2019_='0.35968960732115685', Health_Behaviors_rank='1.7164754505547513', House_per_sq_mile='11.836807578091502', ICU_Beds='43.5335293961527', Length_of_Life_rank='1.7931523467720412', MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_='2.4811040685325825', Non_profit_hospital_beds_per_1000_people__2019_='1.0993748666024383', Number_of_Tests_with_Results_per_1_000_Population='1.918537174883447', POP_ESTIMATE_2018='13.73768721011897', Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_='5.501828328967119', Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_='1.6427684491488883', Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_='0.5060181629157671', Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_='-0.2359653309914734', Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_='0.24678413845870986', Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_='-0.7317118973619294', Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_='-0.1000231220302085', Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_='0.47600978309285336', Physical_Environment_rank='1.7102498563305553', Population_per_sq_mile='11.066295533001421', Quality_of_Life_rank='1.7272797185486595', Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_='0.5621054363577925', Share_of_Tests_with_Positive_COVID_19_Results='0.9348694979526109', Social___Economic_Factors_rank='1.7412612938288288', State_Local_Government_hospital_beds_per_1000_people__2019_='2.408457697241497', Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_='2.7731190701271093', Surgery_specialists__2019_='1.3250169644535321', Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_='1.5192576017131847', Total_Hospitals__2019_='1.4215019639257556', Total_Primary_Care_Physicians__2019_='1.4089144461352563', Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_='1.876532634631023', Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_='2.231936463898481', Total_Specialist_Physicians__2019_='2.034446003527229', Total_hospital_beds_per_1000_people__2019_='0.8223893999684955', Total_nurse_practitioners__2019_='0.7476873010234218', Total_physician_assistants__2019_='-0.026001829737361298', cases='9.726344345814274', confirmed_cases='11.009453662781933', confirmed_deaths='9.791804720559224', county='None', deaths='5.02635252504621', length_of_lockdown='-0.5743978518845761', pop_fraction='9.584592646351235', state='None', summary='skew', total_state_pop='1.8060966426268625'), Row(Active_General_Surgeons_per_100_000_Population__2018__AAMC_='13.920041750189267', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='4.171836511502543', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='4.2371526300850055', Active_Physicians_per_100_000_Population__2018__AAMC_='14.281041742431455', Active_Pr

In [17]:
zeroinf_analysis_described.collect()

Out[41]: [Row(summary='count', county='2890', confirmed_cases='2890', confirmed_deaths='2890', state='2890', length_of_lockdown='2890', cases='2890', deaths='2890', POP_ESTIMATE_2018='2890', total_state_pop='2890', Active_Physicians_per_100_000_Population__2018__AAMC_='2890', Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_='2890', Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='2890', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='2890', Active_General_Surgeons_per_100_000_Population__2018__AAMC_='2890', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='2890', Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_='2890', Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_='2890', Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_='2890', MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_='2890', Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_='2890', Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_='2890', Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_='2890', Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_='2890', Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_='2890', Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_='2890', Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_='2890', Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_='2890', Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_='2890', All_Specialties__AAMC_='2890', State_Local_Government_hospital_beds_per_1000_people__2019_='2890', Non_profit_hospital_beds_per_1000_people__2019_='2890', For_profit_hospital_beds_per_1000_people__2019_='2890', Total_hospital_beds_per_1000_people__2019_='2890', Total_nurse_practitioners__2019_='2890', Total_physician_assistants__2019_='2890', Total_Hospitals__2019_='2890', Total_Primary_Care_Physicians__2019_='2890', Surgery_specialists__2019_='2890', Emergency_Medicine_specialists__2019_='2890', Total_Specialist_Physicians__2019_='2890', ICU_Beds='2890', pop_fraction='2890', Length_of_Life_rank='2890', Quality_of_Life_rank='2890', Health_Behaviors_rank='2890', Clinical_Care_rank='2890', Social___Economic_Factors_rank='2890', Physical_Environment_rank='2890', Adult_smoking_percentage='2890', Adult_obesity_percentage='2890', Excessive_drinking_percentage='2890', Population_per_sq_mile='2890', House_per_sq_mile='2890', Share_of_Tests_with_Positive_COVID_19_Results='2890', Number_of_Tests_with_Results_per_1_000_Population='2890'),
 Row(summary='mean', county=None, confirmed_cases='375.2373702422145', confirmed_deaths='19.05363321799308', state=None, length_of_lockdown='34.3', cases='2.37343276013253', deaths='0.0986733990647059', POP_ESTIMATE_2018='109859.56712802769', total_state_pop='1.8063285047058824E7', Active_Physicians_per_100_000_Population__2018__AAMC_='255.83636678200773', Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_='226.6442906574413', Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='88.00505190311414', Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_='80.04795847750937', Active_General_Surgeons_per_100_000_Population__2018__AAMC_='7.609515570934332', Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_='6.750311418685208', Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_='33.87982698961921', Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_='20.962802768166316', Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_='31.211107

In [18]:
new_describe = sc.parallelize(new_data).toDF()           #turns the results from our loop into a dataframe
new_describe = new_describe.select(zeroinf_analysis_described.columns) #forces the columns into the same order

expanded_describe = zeroinf_analysis_described.unionAll(new_describe)  #merges the new stats with the original describe
expanded_describe.show()


+--------+---------+------------------+------------------+-------+--------------------+------------------+-------------------+------------------+--------------------+-----------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------+-------------------------------------------------------------------------------+-----------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------------------+------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+----------------------+-----------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+------------------------------------------+--------------------------------+---------------------------------+----------------------+------------------------------------+--------------------------+-------------------------------------+----------------------------------+-------------------+--------------------+-------------------+--------------------+---------------------+------------------+------------------------------+-------------------------+------------------------+------------------------+-----------------------------+----------------------+------------------+---------------------------------------------+-------------------------------------------------+
 summary| county| confirmed_cases| confirmed_deaths| state| length_of_lockdown| cases| deaths| POP_ESTIMATE_2018| total_state_pop|Active_Physicians_per_100_000_Population__2018__AAMC_|Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_|Active_General_Surgeons_per_100_000_Population__2018__AAMC_|Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_|Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_|MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_|Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_|Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_|Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_|Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_|Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_|Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_|Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_|Percent_Change_in_Residents_and_Fellows_in_ACG

In [19]:
import pandas as pd
#import matplotlib.pyplot as plt

#%matplotlib inline #tells the Jupyter Notebook to display graphs inline (rather than in a separate window)

In [20]:
confirmed_deaths = zeroinf_analysis[['confirmed_deaths']].collect()
confirmed_cases = zeroinf_analysis[['confirmed_cases']].collect()

In [21]:
print(confirmed_deaths[:5])
print(confirmed_cases[:5])

[Row(confirmed_deaths=4), Row(confirmed_deaths=7), Row(confirmed_deaths=1), Row(confirmed_deaths=1), Row(confirmed_deaths=0)]
[Row(confirmed_cases=93), Row(confirmed_cases=231), Row(confirmed_cases=69), Row(confirmed_cases=46), Row(confirmed_cases=45)]

In [22]:
zeroinf_analysis.columns


Out[46]: ['county',
 'confirmed_cases',
 'confirmed_deaths',
 'state',
 'length_of_lockdown',
 'cases',
 'deaths',
 'POP_ESTIMATE_2018',
 'total_state_pop',
 'Active_Physicians_per_100_000_Population__2018__AAMC_',
 'Total_Active_Patient_Care_Physicians_per_100_000_Population__2018__AAMC_',
 'Active_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_',
 'Active_Patient_Care_Primary_Care_Physicians_per_100_000_Population__2018__AAMC_',
 'Active_General_Surgeons_per_100_000_Population__2018__AAMC_',
 'Active_Patient_Care_General_Surgeons_per_100_000_Population__2018__AAMC_',
 'Percentage_of_Active_Physicians_Who_Are_Female__2018__AAMC_',
 'Percentage_of_Active_Physicians_Who_Are_International_Medical_Graduates__IMGs___2018__AAMC_',
 'Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older__2018__AAMC_',
 'MD_and_DO_Student_Enrollment_per_100_000_Population__AY_2018_2019__AAMC_',
 'Student_Enrollment_at_Public_MD_and_DO_Schools_per_100_000_Population__AY_2018_2019__AAMC_',
 'Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools__2008_2018__AAMC_',
 'Percentage_of_MD_Students_Matriculating_In_State__AY_2018_2019__AAMC_',
 'Total_Residents_Fellows_in_ACGME_Programs_per_100_000_Population_as_of_December_31__2018__AAMC_',
 'Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100_000_Population_as_of_Dec__31__2018__AAMC_',
 'Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31__2018__AAMC_',
 'Ratio_of_Residents_and_Fellows__GME__to_Medical_Students__UME___AY_2017_2018__AAMC_',
 'Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs__2008_2018__AAMC_',
 'Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education__UME___2018__AAMC_',
 'All_Specialties__AAMC_',
 'State_Local_Government_hospital_beds_per_1000_people__2019_',
 'Non_profit_hospital_beds_per_1000_people__2019_',
 'For_profit_hospital_beds_per_1000_people__2019_',
 'Total_hospital_beds_per_1000_people__2019_',
 'Total_nurse_practitioners__2019_',
 'Total_physician_assistants__2019_',
 'Total_Hospitals__2019_',
 'Total_Primary_Care_Physicians__2019_',
 'Surgery_specialists__2019_',
 'Emergency_Medicine_specialists__2019_',
 'Total_Specialist_Physicians__2019_',
 'ICU_Beds',
 'pop_fraction',
 'Length_of_Life_rank',
 'Quality_of_Life_rank',
 'Health_Behaviors_rank',
 'Clinical_Care_rank',
 'Social___Economic_Factors_rank',
 'Physical_Environment_rank',
 'Adult_smoking_percentage',
 'Adult_obesity_percentage',
 'Excessive_drinking_percentage',
 'Population_per_sq_mile',
 'House_per_sq_mile',
 'Share_of_Tests_with_Positive_COVID_19_Results',
 'Number_of_Tests_with_Results_per_1_000_Population']

In [23]:
from pyspark.sql.functions import col

dataset = zeroinf_analysis.select( col('length_of_lockdown'), col('confirmed_cases'), col('confirmed_deaths'), col('POP_ESTIMATE_2018'), col('ICU_Beds'), col('Adult_obesity_percentage'),col('Quality_of_Life_rank'), col('Excessive_drinking_percentage'),col('Population_per_sq_mile'), col('Clinical_Care_rank'), col('Adult_smoking_percentage'), col('Total_Specialist_Physicians__2019_'), col('Physical_Environment_rank'), col('Number_of_Tests_with_Results_per_1_000_Population'))

In [24]:

dataset.show(5)

+------------------+---------------+----------------+-----------------+-----------+------------------------+--------------------+-----------------------------+----------------------+------------------+------------------------+----------------------------------+-------------------------+-------------------------------------------------+
length_of_lockdown|confirmed_cases|confirmed_deaths|POP_ESTIMATE_2018| ICU_Beds|Adult_obesity_percentage|Quality_of_Life_rank|Excessive_drinking_percentage|Population_per_sq_mile|Clinical_Care_rank|Adult_smoking_percentage|Total_Specialist_Physicians__2019_|Physical_Environment_rank|Number_of_Tests_with_Results_per_1_000_Population|
+------------------+---------------+----------------+-----------------+-----------+------------------------+--------------------+-----------------------------+----------------------+------------------+------------------------+----------------------------------+-------------------------+-------------------------------------------------+
 26| 93| 4| 55601|0.107911728| 33| 11.0| 15| 91.8| 13.0| 18| 1.297497418| 53.0| 27.8|
 26| 231| 7| 218022|0.233921347| 31| 2.0| 18| 114.6| 6.0| 17| 1.297497418| 5.0| 27.8|
 26| 69| 1| 24881|0.200956553| 42| 55.0| 13| 31.0| 41.0| 22| 1.297497418| 28.0| 27.8|
 26| 46| 1| 22400| 0.0| 38| 14.0| 16| 36.8| 18.0| 19| 1.297497417| 22.0| 27.8|
 26| 45| 0| 57840| 0.10373444| 34| 13.0| 14| 88.9| 45.0| 19| 1.297497418| 49.0| 27.8|
+------------------+---------------+----------------+-----------------+-----------+------------------------+--------------------+-----------------------------+----------------------+------------------+------------------------+----------------------------------+-------------------------+-------------------------------------------------+
only showing top 5 rows

In [25]:
dataset.dtypes

Out[66]: [('length_of_lockdown', 'int'),
 ('confirmed_cases', 'int'),
 ('confirmed_deaths', 'int'),
 ('POP_ESTIMATE_2018', 'int'),
 ('ICU_Beds', 'double'),
 ('Adult_obesity_percentage', 'int'),
 ('Quality_of_Life_rank', 'double'),
 ('Excessive_drinking_percentage', 'int'),
 ('Population_per_sq_mile', 'double'),
 ('Clinical_Care_rank', 'double'),
 ('Adult_smoking_percentage', 'int'),
 ('Total_Specialist_Physicians__2019_', 'double'),
 ('Physical_Environment_rank', 'double'),
 ('Number_of_Tests_with_Results_per_1_000_Population', 'double')]

In [26]:
from pyspark.sql.functions import col


dataset = dataset.withColumnRenamed('confirmed_deaths', 'label')

In [28]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor, GeneralizedLinearRegression, AFTSurvivalRegression
from pyspark.ml.feature import VectorIndexer

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import DoubleType
 
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder  

In [29]:
valuableColumns = list(dataset.columns)

In [30]:
valuableColumns

Out[70]: ['length_of_lockdown',
 'confirmed_cases',
 'label',
 'POP_ESTIMATE_2018',
 'ICU_Beds',
 'Adult_obesity_percentage',
 'Quality_of_Life_rank',
 'Excessive_drinking_percentage',
 'Population_per_sq_mile',
 'Clinical_Care_rank',
 'Adult_smoking_percentage',
 'Total_Specialist_Physicians__2019_',
 'Physical_Environment_rank',
 'Number_of_Tests_with_Results_per_1_000_Population']

In [31]:
for col in valuableColumns[:-1]:
    # Of cause we can't change immutable values, but we can owerwrite them
    dataset = dataset.withColumn(col+"_d", dataset[col].cast("double"))
    
    
dataset = dataset.fillna(-1., subset=valuableColumns)  
dataset

Out[71]: DataFrame[length_of_lockdown: int, confirmed_cases: int, label: int, POP_ESTIMATE_2018: int, ICU_Beds: double, Adult_obesity_percentage: int, Quality_of_Life_rank: double, Excessive_drinking_percentage: int, Population_per_sq_mile: double, Clinical_Care_rank: double, Adult_smoking_percentage: int, Total_Specialist_Physicians__2019_: double, Physical_Environment_rank: double, Number_of_Tests_with_Results_per_1_000_Population: double, length_of_lockdown_d: double, confirmed_cases_d: double, label_d: double, POP_ESTIMATE_2018_d: double, ICU_Beds_d: double, Adult_obesity_percentage_d: double, Quality_of_Life_rank_d: double, Excessive_drinking_percentage_d: double, Population_per_sq_mile_d: double, Clinical_Care_rank_d: double, Adult_smoking_percentage_d: double, Total_Specialist_Physicians__2019__d: double, Physical_Environment_rank_d: double]

In [32]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [33]:
dataset=dataset.rdd.map(lambda x:(Vectors.dense(x[0:-1]), x[-1])).toDF(["features", "label"])
dataset.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[26.0,93.0,4.0,55...| 53.0|
[26.0,231.0,7.0,2...| 5.0|
[26.0,69.0,1.0,24...| 28.0|
[26.0,46.0,1.0,22...| 22.0|
[26.0,45.0,0.0,57...| 49.0|
[26.0,28.0,1.0,10...| 19.0|
[26.0,230.0,6.0,1...| 45.0|
[26.0,127.0,3.0,1...| 37.0|
[26.0,324.0,22.0,...| 40.0|
[26.0,24.0,0.0,26...| 27.0|
[26.0,73.0,1.0,44...| 31.0|
[26.0,72.0,2.0,12...| 23.0|
[26.0,65.0,1.0,23...| 11.0|
[26.0,27.0,1.0,13...| 62.0|
[26.0,13.0,1.0,14...| 8.0|
[26.0,158.0,1.0,5...| 10.0|
[26.0,74.0,2.0,54...| 35.0|
[26.0,16.0,0.0,12...| 25.0|
[26.0,33.0,1.0,10...| 61.0|
[26.0,56.0,1.0,36...| 7.0|
+--------------------+-----+
only showing top 20 rows

In [34]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dataset)

In [35]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3])

In [36]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [37]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

In [38]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)


In [39]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+-----------------+-----------+--------------------+
 prediction| label| features|
+-----------------+-----------+--------------------+
50.33319380957628|49.47965217|[0.0,1.0,0.0,1392...|
2.457627118644068| 3.0|[0.0,1.0,0.0,1445...|
50.33319380957628|49.47965217|[0.0,1.0,0.0,1952...|
6.897637795275591| 8.0|[0.0,1.0,0.0,2153...|
41.40983606557377| 40.0|[0.0,1.0,0.0,2213...|
+-----------------+-----------+--------------------+
only showing top 5 rows

In [40]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 5.4169

In [41]:
treeModel = model.stages[1]
# summary only
print(treeModel)

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_2dfa87bed4af) of depth 5 with 63 nodes

In [42]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [43]:
grid = ParamGridBuilder() \
  .addGrid(dt.maxDepth, [2, 3, 4, 5, 6, 7, 8]) \
  .addGrid(dt.maxBins, [2, 4, 8]) \
  .build()

In [44]:
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=grid, numFolds=3)

In [45]:
# Explicitly create a new run.
# This allows this cell to be run multiple times.
# If you omit mlflow.start_run(), then this cell could run once,
# but a second run would hit conflicts when attempting to overwrite the first run.
import mlflow
import mlflow
import mlflow.mleap
import pyspark
#import pyspark.ml.mleap.SparkUtil 
#import mlflow.mleap.SparkUtil 
import mlflow.mleap
with mlflow.start_run():
  cvModel = cv.fit(trainingData)
  mlflow.set_tag('owner_team', 'UX Data Science') # Logs user-defined tags
  test_metric = evaluator.evaluate(cvModel.transform(testData))
  mlflow.log_metric('testData_' + evaluator.getMetricName(), test_metric) # Logs additional metrics
  mlflow.mleap.log_model(spark_model=cvModel.bestModel, sample_input=testData, artifact_path='dbfs:/databricks/mlflow/2835302286394144') # Logs the best model via mleap


MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [46]:
import mlflow
import mlflow.mleap
from mlflow import log_metric, log_param, log_artifacts
def fit_model():
  import mlflow
  import mlflow.mleap
  from mlflow import log_metric, log_param, log_artifacts
  # Start a new MLflow run
  with mlflow.start_run() as run:
    # Fit the model, performing cross validation to improve accuracy
    grid = ParamGridBuilder().addGrid(dt.maxDepth, [2, 3, 4, 5, 6, 7, 8]).addGrid(dt.maxBins, [2, 4, 8]).build()
    #paramGrid = ParamGridBuilder().addGrid(hashingTF.numFeatures, [1000, 2000]).build()
    cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=grid, numFolds=3)
    #cv = CrossValidator(estimator=pipeline, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid)
    cvModel = cv.fit(trainingData)
    #cvModel = cv.fit(df)
    model = cvModel.bestModel
  
    # Log the model within the MLflow run
    mlflow.mleap.log_model(spark_model=model, sample_input=trainingData, artifact_path="dbfs:/databricks/mlflow/2835302286394144")

In [47]:
fit_model()

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.